## Challenge 1

This colab notebook is used to allow creating an evaluation of your algorithm developed for Challenge 1 and a file to be submitted on "Codalab" the leaderboard

https://codalab.lisn.upsaclay.fr/competitions/11115?secret_key=e47c9134-ebbc-4132-8552-06b8fa3e3df3

Use your algorithm to modify the code below, it will train on 25 pre-selected seeds and produce test output. You can then upload this to see your score in the leaderboard. 

Note the ouput file "submission.zip" is created inside the colab, click on the folder icon on the left to see it and download it. This is the file you submit on the leaderboard.

In order for a top score to count you must also add to the zip file your version of this notebook (so that it can be inspected for violations of the contest rules, entries violating the rules will be removed from the leaderboard).

Violation of contest rules and spirit include for example:
- trying to label the test data in other ways
- violation challenge 1 rules (e.g. using pre-trained models)

### Downloads

Dataset Files:

- Training splits (contain targets): https://drive.google.com/file/d/1-Olha5krpVYKOy7AIPsn20lkQftSVrDy/view?usp=share_link
- Testing splits (no labels): https://drive.google.com/file/d/1-GLipKH63MaTXj0zLqtNtMDKpFGEasB_/view?usp=share_link


Run the following pair of cells to download the files to your session.

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown

file_ids = {'train_data.zip': '1-Olha5krpVYKOy7AIPsn20lkQftSVrDy',
            'test_data.zip': '1-GLipKH63MaTXj0zLqtNtMDKpFGEasB_'}
url_template = 'https://drive.google.com/uc?id={file_id}'

for filename, file_id in file_ids.items():
    url = url_template.format(file_id=file_id)
    gdown.download(url, filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-Olha5krpVYKOy7AIPsn20lkQftSVrDy
To: /content/train_data.zip
100%|██████████| 15.4M/15.4M [00:00<00:00, 49.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-GLipKH63MaTXj0zLqtNtMDKpFGEasB_
To: /content/test_data.zip
100%|██████████| 307M/307M [00:10<00:00, 29.0MB/s]


## Imports

In [ ]:
import os
import re
import zipfile
import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from numpy.random import RandomState
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms

- Import dataset and define preprocessing functions

In [ ]:
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


## Training Template

- Here we provide some code for you to get started with a baseline

In [ ]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.float()
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data).squeeze()

        loss = F.binary_cross_entropy_with_logits(output, target)
        loss.backward()
        optimizer.step()

        pred = output > 0.5 
        correct += pred.eq(target.view_as(pred)).sum().item()
    if display:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
    return correct / len(train_loader.dataset)

def predict(model,
            test_data,
            device=None,
            data_transform=transform_val,
            data_desc='prediction set'):
    device = torch.device(device or "cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions = []
    with torch.no_grad():
        for data in tqdm(test_data, desc=data_desc):
            data = torch.tensor(data).to(device)
            output = model(data)
            pred = 1 if output.item() > 0.5 else 0
            predictions.append(pred)
    return predictions

In [97]:
# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.layers = nn.ModuleList()
        
#         self.layers+=[nn.Conv2d(3, 16,  kernel_size=3) , 
#                       nn.ReLU(inplace=True)]
#         self.layers+=[nn.Conv2d(16, 16,  kernel_size=3, stride=2), 
#                       nn.ReLU(inplace=True)]
#         self.layers+=[nn.Conv2d(16, 32,  kernel_size=3), 
#                       nn.ReLU(inplace=True)]
#         self.layers+=[nn.Conv2d(32, 32,  kernel_size=3, stride=2), 
#                       nn.ReLU(inplace=True)]
#         self.fc = nn.Linear(32*5*5, 1)
#     def forward(self, x):
#         for i in range(len(self.layers)):
#           x = self.layers[i](x)
#         x = x.view(-1, 32*5*5)
#         x = self.fc(x)
#         return x

class Net(torch.nn.Module):
    def __init__(self,input_size,hidden_size):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
                        # nn.Conv2d(64*2,64,kernel_size=3,padding=0),
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        

        
# self.fc1 = nn.Linear(128*2*2,hidden_size)
        
      
        # self.fc1 = nn.Linear(input_size*3*3,hidden_size)
        self.fc1 = nn.Linear(2304,hidden_size)
        self.fc2 = nn.Linear(hidden_size,1)

    def forward(self,x):
        if len(x.shape)<4:
          # print('TRUE')
          x  = x.unsqueeze(0)
        out = self.layer1(x)
        # print(f'dim layer 1 = {out.shape}')
        out = self.layer2(out)
        # print(f'dim layer 2 = {out.shape}')
#
        # out = self.layer3(out)
        # print(f'dim layer 3 = {out.shape}')
        
        out = out.view(out.size(0),-1)
        # print(f'dim flatten = {out.shape}')
        out = F.relu(self.fc1(out))
        # print(f'dim fc1 = {out.shape}')

        out = F.sigmoid(self.fc2(out))
        return out

- The cells below assumes you have uploaded the provided `train_data.zip` file which contains the given training splits. It will validate on random samples.

    Keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [98]:
# Load datasets
train_splits = []
train_targets = []
with zipfile.ZipFile('train_data.zip', mode='r') as archive:
    for a in archive.namelist():
        with archive.open(a) as f:
            d = np.load(f, allow_pickle=True)
            train_splits.append(d[0])
            train_targets.append(d[1])

test_splits = []
with zipfile.ZipFile('test_data.zip', mode='r') as archive:
    for a in archive.namelist():
        with archive.open(a) as f:
            d = np.load(f, allow_pickle=True)
            test_splits.append(d)

- You may use the following script to run your training experiments.

In [99]:
models = []
train_accuracies = []
eval_predictions = []
for i, (train_X, train_Y, test_X) in enumerate(zip(tqdm(train_splits,
                                                        desc='seeds loop'),
                                                   train_targets,
                                                   test_splits)):
    ## --- START of model definition ---
    ## EDIT the following lines with your own
    ## model/optimizer/[scheduler] implementation.
    ## (here we just used the baseline provided above)
    config = dict(batch_size=16,
                  epochs=250,
                  learning_rate=1e-3,
                  momentum=0.9,
                  weight_decay=5e-4)
    # model = Net(16,32).to(device)
    model = Net(16,8).to(device)
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=config['learning_rate'],
                                momentum=config['momentum'],
                                weight_decay=config['weight_decay'])
    ## --- END of model definition ---
    
    # load train split
    train_loader = DataLoader(TensorDataset(train_X, train_Y),
                              batch_size=config['batch_size'],
                              shuffle=True)
    
    # do training
    for epoch in tqdm(range(config['epochs']), desc=f'training model {i+1}'):
        acc = train(model, device, train_loader,
                    optimizer, epoch,
                    display=epoch%(config['epochs']//4)==0)
    print(f'final train acc for model {i+1}: {acc:.2%}')
    train_accuracies.append(acc)
    models.append(model)

    # evaluate model
    pred_Y = predict(model, test_X, 
                     data_desc=f'CODALab eval instance {i+1}')
    eval_predictions.append(pred_Y)

print(f'Mean Train Acc over {len(train_splits)} models: '\
      f'{np.mean(train_accuracies):.2%} '\
      f'+- {np.std(train_accuracies):.2}')

seeds loop:   0%|          | 0/25 [00:00<?, ?it/s]

training model 1:   0%|          | 0/250 [00:00<?, ?it/s]

Train Epoch: 0 [6/50 (75%)]	Loss: 0.734554
Train Epoch: 62 [6/50 (75%)]	Loss: 0.331944
Train Epoch: 124 [6/50 (75%)]	Loss: 0.698877
Train Epoch: 186 [6/50 (75%)]	Loss: 0.331489
Train Epoch: 248 [6/50 (75%)]	Loss: 0.503916
final train acc for model 1: 100.00%


CODALab eval instance 1:   0%|          | 0/1000 [00:00<?, ?it/s]

training model 2:   0%|          | 0/250 [00:00<?, ?it/s]

...

CODALab eval instance 24:   0%|          | 0/1000 [00:00<?, ?it/s]

training model 25:   0%|          | 0/250 [00:00<?, ?it/s]

Train Epoch: 0 [6/50 (75%)]	Loss: 0.730605
Train Epoch: 62 [6/50 (75%)]	Loss: 0.755414
Train Epoch: 124 [6/50 (75%)]	Loss: 0.330483
Train Epoch: 186 [6/50 (75%)]	Loss: 0.698478
Train Epoch: 248 [6/50 (75%)]	Loss: 0.332055
final train acc for model 25: 100.00%


CODALab eval instance 25:   0%|          | 0/1000 [00:00<?, ?it/s]

Mean Train Acc over 25 models: 99.92% +- 0.0039


## Test and Submit to CODALab

**RUN THE FOLLOWING CELL** to run the testing loop. 

Results are saved using the CODALab path convention, and zipped at the end
    
Download the generated `submission.zip` file and upload it to CODALab.

You may _only_ change the `model` argument of `run_testing` to test your own neural network. (i.e., if you change anything else your submission might crash in CODALab!)

In [101]:
def zip_evaluation(models,
                   test_data_location = 'test_data',
                   test_data_file='test_data.zip',
                   submission_file='submission.zip',
                   predpath='./submission/input/res',
                   verbose=False):
    verb = lambda *s: print('[INFO] ', *s) if verbose else None
    
    verb(f'Files will be inside {predpath}')
    if not os.path.exists(predpath):
        os.makedirs(predpath)

    verb(f'Compressing in {submission_file}')
    with zipfile.ZipFile(submission_file, mode='a') as archive:
        for i, pred_Y in tqdm(enumerate(eval_predictions),
                              total=len(eval_predictions),
                              desc='seed loop'):
            np.savetxt(f'{predpath}/predictions_{i}.txt', pred_Y)
            archive.write(f'{predpath}/predictions_{i}.txt')
    verb('All Done Successfully :)')

zip_evaluation(model, verbose=True)

[INFO]  Files will be inside ./submission/input/res
[INFO]  Compressing in submission.zip


seed loop:   0%|          | 0/25 [00:00<?, ?it/s]

[INFO]  All Done Successfully :)
